### Consulta 1
def userdata( User_id : str ): Debe devolver cantidad de dinero gastado por el usuario, el porcentaje de recomendación en base a reviews.recommend y cantidad de items.


In [107]:
import pandas as pd

In [108]:
columns = ['id', 'price']
games_price = pd.read_csv(r'./datasets/steam_games.csv', usecols=columns)
games_price.head()

,price,id
0,4.99,761140.0
1,Free To Play,643980.0
2,Free to Play,670290.0
3,0.99,767400.0
4,2.99,773570.0


In [109]:
games_price.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32135 entries, 0 to 32134
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   price   30758 non-null  object 
 1   id      32133 non-null  float64
dtypes: float64(1), object(1)
memory usage: 502.2+ KB


In [110]:
unique_list = []
for i in games_price['price']:
    try:
        float(i)
    except:
        unique_list.append(i)

uniques = set(unique_list)
uniques_not_free = ['Starting at $499.00', 'Starting at $449.00']

In [111]:
print(uniques)

{'Free to Try', 'Free to Play', 'Play for Free!', 'Play WARMACHINE: Tactics Demo', 'Play Now', 'Free HITMAN™ Holiday Pack', 'Free To Play', 'Play the Demo', 'Starting at $499.00', 'Third-party', 'Free Movie', 'Starting at $449.00', 'Free', 'Install Now', 'Free Demo', 'Install Theme', 'Free to Use', 'Free Mod'}


In [112]:
games_price['price_fixed'] = games_price['price'].apply(lambda x: 0 if x in uniques 
                                                        else 499.0 if x=='Starting at $499.00'
                                                        else 449.0 if x=='Starting at $449.00'
                                                        else x)
games_price.head()

,price,id,price_fixed
0,4.99,761140.0,4.99
1,Free To Play,643980.0,0
2,Free to Play,670290.0,0
3,0.99,767400.0,0.99
4,2.99,773570.0,2.99


In [113]:
games_price['price_fixed'] = games_price['price_fixed'].astype(float)

In [114]:
games_price.rename(columns={'id':'item_id'},inplace=True)
games_price.drop(columns='price', inplace=True)
games_price.head()

,item_id,price_fixed
0,761140.0,4.99
1,643980.0,0.00
2,670290.0,0.00
3,767400.0,0.99
4,773570.0,2.99


In [115]:
games_price.isna().sum()

item_id           2
price_fixed    1377
dtype: int64

In [116]:
games_price.fillna(0, inplace=True)

In [117]:
games_price.isna().sum()

item_id        0
price_fixed    0
dtype: int64

In [118]:
columns = ['user_id', 'item_id', 'items_count']
user_items = pd.read_csv('datasets/aus_users_items.csv', usecols=columns)
user_items.head()

,user_id,items_count,item_id
0,76561197970982479,277,10
1,76561197970982479,277,20
2,76561197970982479,277,30
3,76561197970982479,277,40
4,76561197970982479,277,50


In [119]:
df_merged = user_items.merge(games_price, on='item_id', how='left')
df_merged.sample(10)

,user_id,items_count,item_id,price_fixed
856758,76561198008592438,294,28050,NaN
2614458,76561198064945689,155,108800,29.99
1430116,Victorypower,217,264240,9.99
5089894,76561198101205256,144,218230,0.00
4554080,76561198058959788,56,340,0.00
2974978,091263,369,70000,9.99
264034,76561198051309911,28,38410,9.99
2892329,nolongerliving,604,22230,9.99
191383,76561198080592688,242,411190,NaN
3343785,76561198079524591,66,268850,0.00


In [120]:
user_spent = df_merged.groupby('user_id')['price_fixed'].agg('sum')
user_spent = pd.DataFrame(user_spent)
user_spent.reset_index()

,user_id,price_fixed
0,--000--,402.77
1,--ace--,166.82
2,--ionex--,109.92
3,-2SV-vuLB-Kg,437.49
4,-404PageNotFound-,1514.31
...,...,...
70907,zzonci,19.98
70908,zzoptimuszz,64.98
70909,zzydrax,99.94
70910,zzyfo,828.51


In [121]:
columns = ['user_id', 'recommend']
user_revs = pd.read_csv('./datasets/aus_user_revs_clean.csv', usecols=columns)



In [122]:
user_revs.sample(5)

,user_id,recommend
6602,76561198140052687,True
4990,Revolver_,True
31248,76561198157987157,True
58257,76561198115910259,True
51946,Katzieon,False


In [123]:
user_gb_count_revs = pd.DataFrame()

user_gb_count_revs['count_total'] = user_revs.groupby('user_id')['recommend'].agg('count')
user_gb_count_revs['count_true'] = user_revs[user_revs['recommend'] == True].groupby('user_id')['recommend'].agg(count_revs='count')


user_gb_count_revs.reset_index(inplace=True)
user_gb_count_revs

# proporcion
user_gb_count_revs['reco_porcentaje'] = user_gb_count_revs['count_true'] / user_gb_count_revs['count_total']
user_gb_count_revs.fillna(0,inplace=True)

In [124]:
user_gb_count_revs.sample(5)

,user_id,count_total,count_true,reco_porcentaje
22561,mylordbendtner,3,2.0,0.666667
7104,76561198072813477,1,1.0,1.000000
17043,SuddenNight,1,1.0,1.000000
24845,umicornbl,1,1.0,1.000000
371,45514551,4,4.0,1.000000


In [125]:
user_revs[user_revs['user_id']=='iliakitmail']

,user_id,recommend
27618,iliakitmail,False


In [127]:
df_merge2 = user_gb_count_revs.merge(user_spent, on='user_id',how='left' )
df_merge2.rename(columns={'price_fixed':'total_spent'},inplace=True)


In [136]:
df_merge3 = df_merge2.merge(user_items[['user_id','items_count']].drop_duplicates(),on='user_id',how='left')
df_merge3 = df_merge3.drop(['count_total','count_true'],axis=1)
df_merge3['reco_porcentaje'] = df_merge3['reco_porcentaje'].apply(lambda x : int(x *100))
df_merge3.rename(columns={'reco_porcentaje':'procentaje_recomendacion', 'total_spent':'cantidad_gastada', 'items_count':'cantidad_items'},inplace=True)

In [137]:
df_merge3.to_csv(r'dataquery/user_data.csv',index=False)

In [138]:
def userdata(user_id : str):
    df_user = pd.read_csv('dataquery/user_data.csv')
    user_data = df_user[df_user['user_id'] == user_id]
    
    return user_data.to_json(orient='records')

In [139]:
userdata('--000--')

'[{"user_id":"--000--","procentaje_recomendacion":100,"cantidad_gastada":402.77,"cantidad_items":58.0}]'